In [1]:

from collections import defaultdict
import numpy as np
import pandas
import scipy.io
from sklearn.metrics import average_precision_score
import sys

sys.path.insert(0, '../Common/')
from Dataset import ImageDataset, my_dataloader
from ModelWrapper import ModelWrapper
from ResNet import get_model

unrel = '/home/gregory/Desktop/unrel'


In [2]:
# Get info for the UnRel dataset

annotations = scipy.io.loadmat('{}/annotations.mat'.format(unrel))['annotations']

files = []
label_dict = defaultdict(list)

for a in annotations:
    
    # len(o) = 4
    # o[0] -> filename
    # o[2][0][0][0][0] -> info about objects, their locations, and relationship
    o = a[0][0][0]
    y = o[2][0][0][0][0]
    
    file = '{}/images/{}'.format(unrel, o[0][0])
    files.append(file)
    
    label_dict[y[0][0]].append(file)
    label_dict[y[1][0]].append(file)
     
classes = list(label_dict)

# Get the object pairs that SPIRE identified as representing spurious patterns

pairs = pandas.read_csv('./0-FindPairs/Pairs.csv')
classes_spire = list(pairs['Main'])

# Find the shared classes

shared = np.intersect1d(classes, classes_spire)
print(shared)

['bench' 'dog' 'skateboard' 'tie' 'truck']


In [3]:
configs = [('bench', 15), ('dog', 18), ('skateboard', 41), ('tie', 32), ('truck', 8)]
modes = {'initial-tune': 'Baseline', 'spire': 'SPIRE', 'fs-3': 'FS'}
trials = [0,1,2,3,4,5,6,7]

out = {}

print('Results per object')

for config in configs:
    
    label = config[0]
    index = config[1]
    
    print()
    print('Object: ', label)

    y = []
    for file in files:
        if file in label_dict[label]:
            y.append([1])
        else:
            y.append([0])

    y = np.array(y, dtype = np.float32)
    
    out_config = {}

    for mode in modes:
        
        tmp = []

        for trial in trials:  

            model_dir = './2-Models/Models/{}/trial{}/model.pt'.format(mode, trial)

            model, _ = get_model(mode = 'tune', parent = model_dir, out_features = 91)
            model.eval()
            model.cuda();

            wrapper = ModelWrapper(model)

            dataset = ImageDataset(files, y)
            dataloader = my_dataloader(dataset)
            y_hat, y_true = wrapper.predict_dataset(dataloader)

            v = average_precision_score(y_true, y_hat[:, index])
            
            tmp.append(v)

        print(modes[mode], np.round(np.mean(tmp), 3), np.round(np.std(tmp), 4))
        out_config[mode] = tmp
    out[config] = out_config

print()
print()
print('Aggregated results')
print()

for mode in modes:
    tmp = []
    for label in out:
        tmp.append(np.mean(out[label][mode]))
    print(modes[mode], np.round(np.mean(tmp), 3))

Results per object

Object:  bench
Baseline 0.457 0.0655
SPIRE 0.455 0.0472
FS 0.452 0.0206

Object:  dog
Baseline 0.86 0.0161
SPIRE 0.852 0.0095
FS 0.84 0.0166

Object:  skateboard
Baseline 0.098 0.0534
SPIRE 0.152 0.0433
FS 0.139 0.0499

Object:  tie
Baseline 0.16 0.0573
SPIRE 0.213 0.0662
FS 0.188 0.0623

Object:  truck
Baseline 0.37 0.0585
SPIRE 0.391 0.0531
FS 0.363 0.0341


Aggregated results

Baseline 0.389
SPIRE 0.413
FS 0.396
